<a href="https://colab.research.google.com/github/DrStephPowers/BIA/blob/main/Ch9_Resume_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting up Textract

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install "setuptools<58.0.0"

It will ask you to restart session. Click okay. Then run again from the top.

In [ ]:
!pip install textract

It will ask you to restart session. Click okay. Then run again from the top.

In [ ]:
!pip install pdfminer.six

### Import Resumes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import textract

In [ ]:
P1 = textract.process("/content/drive/MyDrive/Datasets/Resumes/Person01.pdf", method='pdfminer',
    language='eng')
P1 = P1.decode("utf-8")

### Clean Text

In [ ]:
import re

In [ ]:
def clean_text(text):
    text = re.sub('\xa0', ' ', text)
    text = re.sub('\n•', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\ue080', ' ', text)
    text = re.sub('\x0c', ' ', text)
    html = re.compile('[<,#*?>]')
    text = html.sub(r' ',text)
    url = re.compile('https?://\S+|www\.S+')
    text = url.sub(r' ',text)
    email = re.compile('[A-Za-z0-2]+@[\w]+.[\w]+')
    text = email.sub(r' ',text)
    return text

In [ ]:
p1 = clean_text(P1)

### Tokenization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [ ]:
def TFIDF_tokenize(text):
    tfidf = TfidfVectorizer(stop_words="english")
    tfidf_mat = tfidf.fit_transform(text)
    words = tfidf.get_feature_names_out(text)
    return tfidf_mat, words

In [ ]:
text = [p1, p2, p3, p4, p5]

In [ ]:
tfidf_mat, words  = TFIDF_tokenize(text)

In [ ]:
df_tf_idf = pd.DataFrame(tfidf_mat.toarray(), columns = words)

### Pipeline (SVD and KMeans)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

In [ ]:
svd = TruncatedSVD(n_components=2)
kmeans = KMeans(n_clusters=2)

pipeline = make_pipeline(svd, kmeans)

pipeline.fit(df_tf_idf)
labels = pipeline.predict(df_tf_idf )

In [ ]:
labels

array([1, 1, 0, 0, 0], dtype=int32)